In [13]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Embedding
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle

with open('test.txt', 'r', encoding='utf-8') as file:
    corpus = file.read()

tokenizer = Tokenizer()
tokenizer.fit_on_texts([corpus])
total_words = len(tokenizer.word_index) + 1

input_sequences = []
for line in corpus.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

max_sequence_length = max([len(seq) for seq in input_sequences])
input_sequences = pad_sequences(input_sequences, maxlen=max_sequence_length, padding='pre')

X = input_sequences[:, :-1]
y = input_sequences[:, -1]

embedding_dim = 64
lstm_units = 64

model = Sequential([
    Embedding(total_words, embedding_dim, input_length=max_sequence_length-1),
    LSTM(lstm_units),
    Dense(total_words, activation='softmax')
])

optimizer = Adam(learning_rate=0.01)
model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

history = model.fit(X, y, epochs=30, verbose=1, batch_size=64)

model.save('text_generation_lstm_model_v2.h5')
with open('tokenizer_v2.pickle', 'wb') as tokenizer_file:
    pickle.dump(tokenizer, tokenizer_file)


Epoch 1/30
29/29 [==============================] - 7s 182ms/step - loss: 6.0805 - accuracy: 0.0596
Epoch 2/30
29/29 [==============================] - 5s 188ms/step - loss: 5.5963 - accuracy: 0.0579
Epoch 3/30
29/29 [==============================] - 5s 188ms/step - loss: 5.2977 - accuracy: 0.0850
Epoch 4/30
29/29 [==============================] - 5s 181ms/step - loss: 4.9914 - accuracy: 0.1010
Epoch 5/30
29/29 [==============================] - 5s 183ms/step - loss: 4.5903 - accuracy: 0.1501
Epoch 6/30
29/29 [==============================] - 6s 216ms/step - loss: 4.1638 - accuracy: 0.1932
Epoch 7/30
29/29 [==============================] - 6s 213ms/step - loss: 3.6979 - accuracy: 0.2517
Epoch 8/30
29/29 [==============================] - 6s 190ms/step - loss: 3.2641 - accuracy: 0.3068
Epoch 9/30
29/29 [==============================] - 5s 189ms/step - loss: 2.8370 - accuracy: 0.3593
Epoch 10/30
29/29 [==============================] - 6s 191ms/step - loss: 2.4533 - accuracy: 0.4299

In [19]:
model = load_model('text_generation_lstm_model_v2.h5')
with open('tokenizer_v2.pickle', 'rb') as tokenizer_file:
    tokenizer = pickle.load(tokenizer_file)

def generate_text(seed_text, next_words, tokenizer, max_sequence_length, model):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_length - 1, padding='pre')
        predicted = model.predict(token_list, verbose=0)
        predicted_word_index = np.argmax(predicted)
        predicted_word = tokenizer.index_word[predicted_word_index]

        seed_text += " " + predicted_word

    return seed_text

seed_text = "as the sun gently nudged awoke it's captives"
next_words = 10

# Generate text
generated_text = generate_text(seed_text, next_words, tokenizer, max_sequence_length, model)
print(generated_text)


as the sun gently nudged awoke it's captives in the hall quickly rushed over to pay their respects
